In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

ticker = 'AAPL'
filename = '../../../../Data_Store/Yah_Daily_Prices/Daily_Price_Data/{}.pk'.format(ticker)
with open(filename, 'rb') as file:
    # Change Index to timestamp for original Polygon Data
    # Change Index to Date for Yah Finance Data
    data = pd.DataFrame(pickle.load(file)).set_index('Date', drop=True)

##### Calculating the EMA requires one more observation than the SMA. Suppose that you want to use 20 days as the number of observations for the EMA. Then, you must wait until the 20th day to obtain the SMA. On the 21st day, you can then use the SMA from the previous day as the first EMA for yesterday.

In [2]:
data = data.fillna(0)
smoothing = 2
multiplier = {}

for period in [12, 26]:
    multiplier["{}".format(period)] = float(smoothing/(1+period))
    data['sma_{}'.format(period)] = data['Adj Close'].rolling(window=period).mean()
    data['ema_{}'.format(period)] = [data['Adj Close'][i]*multiplier["{}".format(period)] + data['sma_{}'.format(period)][i-1]*(1 - multiplier["{}".format(period)]) for i in range(0, len(data))]
    data['ema_{}'.format(period)] = [data['Adj Close'][i]*multiplier["{}".format(period)] + data['ema_{}'.format(period)][i-1]*(1 - multiplier["{}".format(period)]) for i in range(0, len(data))]
   
    
data['MACD'] = data['ema_12'] - data['ema_26']
sig_multiplier = float(2.0/(1+9))
data['sma_MACD'] = data['MACD'].rolling(window=period).mean()
data['signal_line'] = [data['MACD'][i]*sig_multiplier + data['sma_MACD'][i]*(1 - sig_multiplier) for i in range(0, len(data))]
data['signal_line'] = [data['MACD'][i] * sig_multiplier + data['signal_line'][i-1]*(1 - sig_multiplier) for i in range(0, len(data))]

In [3]:
data.head()

,High,Low,Open,Close,Volume,Adj Close,sma_12,ema_12,sma_26,ema_26,MACD,sma_MACD,signal_line
Date,,,,,,,,,,,,,
2000-01-03,1.004464,0.907924,0.936384,0.999442,535796800.0,0.856227,NaN,113.300681,NaN,119.716215,-6.415534,NaN,0.680796
2000-01-04,0.987723,0.903460,0.966518,0.915179,512377600.0,0.784038,NaN,95.968262,NaN,110.975329,-15.007067,NaN,NaN
2000-01-05,0.987165,0.919643,0.926339,0.928571,778321600.0,0.795511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,0.955357,0.848214,0.947545,0.848214,767972800.0,0.726669,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,0.901786,0.852679,0.861607,0.888393,460734400.0,0.761090,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.tail()

,High,Low,Open,Close,Volume,Adj Close,sma_12,ema_12,sma_26,ema_26,MACD,sma_MACD,signal_line
Date,,,,,,,,,,,,,
2021-04-23,135.119995,132.160004,132.160004,134.320007,78657500.0,134.320007,133.119167,132.425485,127.393077,127.692758,4.732726,0.720986,1.915959
2021-04-26,135.059998,133.559998,134.830002,134.720001,66905100.0,134.720001,133.482501,133.138724,127.938846,128.095673,5.043051,1.054784,2.227277
2021-04-27,135.410004,134.110001,135.009995,134.389999,66015800.0,134.389999,133.598334,133.523073,128.492693,128.413900,5.109172,1.394346,2.503784
2021-04-28,135.020004,133.080002,134.309998,133.580002,106589600.0,133.580002,133.793334,133.615637,128.884616,128.799176,4.816461,1.704398,2.673141
2021-04-29,137.070007,132.449997,136.470001,133.479996,150777100.0,133.479996,133.714167,133.577742,129.305385,129.211046,4.366695,1.976924,2.734787
